In [1]:
from qdrant_client import QdrantClient

client = QdrantClient("http://localhost:6333")
client.get_collections()

/home/kantundpeterpan/projects/zoomcamp/zcllm/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CollectionsResponse(collections=[CollectionDescription(name='medicode_csp'), CollectionDescription(name='zoomcamp-sparse'), CollectionDescription(name='zoomcamp-sparse-and-dense')])

In [2]:
from fastembed import TextEmbedding

In [5]:
embedding = TextEmbedding(
    'jinaai/jina-embeddings-v2-small-en'
)

# Q1

In [10]:
query = 'I just discovered the course. Can I join now?'

In [13]:
query_vector = [*embedding.embed(
    query
)][0]
query_vector.min()

np.float64(-0.11726373885183883)

# Q2

In [16]:
import numpy as np

In [12]:
doc = 'Can I still join the course after the start date?'

In [15]:
doc_vector = [*embedding.embed(
    doc
)][0]

In [17]:
np.dot(query_vector, doc_vector)

np.float64(0.9008528895674548)

# Q3

In [18]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

In [19]:
docvecs = [*embedding.embed(
    [d['text'] for d in documents]
)]

In [21]:
cosine_sims = [
    *map(lambda dv: np.dot(query_vector, dv), docvecs)
]

In [23]:
cosine_sims.index(max(cosine_sims))

1

# Q4

In [26]:
full_text = [d['question'] + d['text'] for d in documents] 

In [27]:
full_vecs = np.array([
    *embedding.embed(full_text)
])

In [29]:
cosine_sims_full = full_vecs.dot(query_vector)

In [34]:
list(cosine_sims_full).index(
    cosine_sims_full.max()
)

0

# Q5

In [36]:
import pandas as pd

In [38]:
supported_models = (
    pd.DataFrame(TextEmbedding.list_supported_models())
    .sort_values("size_in_GB")
    .drop(columns=["sources", "model_file", "additional_files"])
    .reset_index(drop=True)
)
supported_models.dim.min()

np.int64(384)

# Q7

In [39]:
import requests 

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()


documents = []

for course in documents_raw:
    course_name = course['course']
    if course_name != 'machine-learning-zoomcamp':
        continue

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [40]:
full_text = [
    d['question'] + d['text'] 
    for d in documents
] 

In [41]:
model = 'BAAI/bge-small-en'

In [43]:
from qdrant_client import models

In [47]:
# Create the collection with both vector types
client.create_collection(
    collection_name="zoomcamp-week2-homework",
    vectors_config={
        # Named dense vector for jinaai/jina-embeddings-v2-small-en
        model: models.VectorParams(
            size=384,
            distance=models.Distance.COSINE,
        ),
    },
)

True

In [50]:
import uuid

In [53]:
client.upsert(
    collection_name="zoomcamp-week2-homework",
    points=[
        models.PointStruct(
            id=uuid.uuid4().hex,
            vector={
                model: models.Document(
                    text=doc,
                    model=model,
                    )
            },
            payload={
                "text": doc,
            }
        )
        for doc in full_text
    ]
)

Fetching 5 files: 100%|██████████| 5/5 [00:01<00:00,  2.76it/s]


UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [57]:
client.query_points(
    collection_name = 'zoomcamp-week2-homework',
    query = models.Document(
        text = query,
        model = model
    ),
    using = model
).points[0].score

0.8703172